In [1]:
from pathlib import Path
from pprint import pprint

import json5

In [2]:
def load_logs_from_file(log_file_path):
    log_file_path = Path(log_file_path)
    if log_file_path.exists():
        with log_file_path.open(encoding="utf-8") as f:
            logs = json5.load(f)
            print("Загруженные логи:")
            pprint(logs, indent=4, width=100)
            return logs
    else:
        print(f"Файл {log_file_path} не найден.")
        return None

In [3]:
log_file_path = "../.cache/method_logs.json"
logs_dict = load_logs_from_file(log_file_path)

Загруженные логи:
[   {   'method_args': {   'init': 'svd',
                           'n_iter_max': 100,
                           'random_state': 42,
                           'rank': [220, 360, 202, 3],
                           'svd': 'truncated_svd'},
        'method_name': 'TensorLy_Tucker_pytorch_truncated_svd_svd',
        'qualitative_metrics': {   'Decomposition method': 'Tucker',
                                   'Language': 'Python',
                                   'Library': 'TensorLy',
                                   'Platform': 'CPU, GPU',
                                   'Tensor type': 'Dense',
                                   'TensorLy backend': 'pytorch'},
        'quantitative_metrics': {   'compression_ratio': 50.27837471247125,
                                    'duration': {   'max': 7.773032903671265,
                                                    'mean': 7.773032903671265,
                                                    'min': 7.773032903